# Multi-agent decentralized speaker selection

This notebook showcases how to implement a multi-agent simulation without a fixed schedule for who speaks when. Instead the agents decide for themselves who speaks. We can implement this by having each agent bid to speak. Whichever agent's bid is the highest gets to speak.

We will show how to do this in the example below that showcases a fictitious presidential debate.

The main difference between simulating two players and multiple players is in revising the schedule for when each agent speaks. To this end, we augment `DialogueSimulator` to take in a custom function that determines the schedule of which agent speaks

## Import Modules 

In [1]:
# Import Libraries
from typing import Callable, List

import numpy as np
import tenacity
from langchain.output_parsers import RegexParser
from langchain.prompts import PromptTemplate
from langchain.schema import (
    HumanMessage,
    SystemMessage,
)

from langchain_openai import ChatOpenAI

# Define Classes
We will use the same `DialogueAgent` and `DialogueSimulator` classes defined in [Multi-Player Dungeons & Dragons](https://python.langchain.com/en/latest/use_cases/agent_simulations/multi_player_dnd.html).

## **`DialogueAgent`**
The **`DialogueAgent`** class is a simple wrapper around the ChatOpenAI model that stores the message history from the dialogue_agent's point of view by simply concatenating the messages as strings.

It exposes two methods:

- send(): applies the chatmodel to the **message history** and returns the message string
- receive(name, message): adds the message spoken by name to **message history**

In [2]:
class DialogueAgent:
    # Dialogue Agent Class
    def __init__(
        self,                           # Initialize the class
        name: str,                      # Name of the agent
        system_message: SystemMessage,  # System message
        model: ChatOpenAI,              # Model
    ) -> None:
        self.name = name
        self.system_message = system_message
        self.model = model
        self.prefix = f"{self.name}: "
        self.reset()

    # Reset the message history
    def reset(self):
        self.message_history = ["Here is the conversation so far."] # Reset the message history

    # Send the message to the model
    def send(self) -> str:
        """
        Applies the chatmodel to the message history
        and returns the message string
        """
        message = self.model.invoke(            # Send the message to the LL model                    
            [
                self.system_message,                                                    
                HumanMessage(content="\n".join(self.message_history + [self.prefix])), 
            ]   # Combine the system message and the message history
        )
        return message.content      # Return the generated message

    # Receive the message from the user > store it in the message history
    def receive(self, name: str, message: str) -> None:
        """
        Concatenates {message} spoken by {name} into message history
        """
        self.message_history.append(f"{name}: {message}")     # Append the message into the message history



## **`DialogueSimulator`**
The **`DialogueSimulator`** class takes a list of agents. At each step, it performs the following:

1. Select the next speaker
2. Calls the next speaker to send a message
3. Broadcasts the message to all other agents
4. Update the step counter. The selection of the next speaker can be implemented as any function, but in this case we simply loop through the agents.

In [3]:
class DialogueSimulator:
    def __init__(
        self,
        agents: List[DialogueAgent],                                    # List of agents
        selection_function: Callable[[int, List[DialogueAgent]], int],  # Select the next speaker
    ) -> None:
        self.agents = agents                        # Initialize the list of agents
        self._step = 0                              # Keep track of the conversation
        self.select_next_speaker = selection_function                   

    # Reset the agents
    def reset(self):
        for agent in self.agents:
            agent.reset()
    
    # Inject the message into the conversation
    def inject(self, name: str, message: str):
        """
        Initiates the conversation with a {message} from {name}
        """
        for agent in self.agents:                   # Loop through the agents, inject the message into the conversation
            agent.receive(name, message)            # Receive the message from each agent

        # increment time
        self._step += 1

    # Step through the conversation
    def step(self) -> tuple[str, str]:
        # 1. choose the next speaker
        speaker_idx = self.select_next_speaker(self._step, self.agents)
        speaker = self.agents[speaker_idx]

        # 2. next speaker sends message
        message = speaker.send()

        # 3. everyone receives message by the speaker
        for receiver in self.agents:
            receiver.receive(speaker.name, message) # Receive the message from the speaker

        # 4. increment time
        self._step += 1

        return speaker.name, message

## `BiddingDialogueAgent`
We define a subclass of `DialogueAgent` that has a `bid()` method that produces a bid given the **message history** and the **most recent message** (Basically extends the *DialogueAgent* class)

In [4]:
# Introduces additional functionality to handle the bidding process
class BiddingDialogueAgent(DialogueAgent):
    def __init__(
        self,
        name,                               # Name of the agent
        system_message: SystemMessage,      # System message provides the context and prompt
        bidding_template: PromptTemplate,   # Bidding template
        model: ChatOpenAI,
    ) -> None:
        super().__init__(name, system_message, model)       # 
        self.bidding_template = bidding_template

    def bid(self) -> str:
        """
        Asks the chat model to output a bid to speak
        """
        prompt = PromptTemplate(
            input_variables=["message_history", "recent_message"],      # Prompt LLM with the message history
            template=self.bidding_template,
        ).format(
            message_history="\n".join(self.message_history),    # define the 'message history'
            recent_message=self.message_history[-1],            # define the 'recent message'
        )
        # Pass the prompt with the message history and get the bid string
        bid_string = self.model.invoke([SystemMessage(content=prompt)]).content
        return bid_string

# Setup Debate

## Define participants and debate topic

In [6]:
character_names = ["Donald Trump", "Kamila Harris", "Xi Jinping"]
topic = "Should the US send weapons to Israel?"
word_limit = 20

## Generate system messages

Basically create prompts and iterates over each 

In [7]:
# Create the system message, basically the debate description
game_description = f"""Here is the topic for the presidential debate: {topic}.
The presidential candidates are: {', '.join(character_names)}."""

player_descriptor_system_message = SystemMessage(
    content="You can add detail to the description of each presidential candidate."
)

# Generate the character descriptions
def generate_character_description(character_name):
    character_specifier_prompt = [
        player_descriptor_system_message,
        HumanMessage(
            content=f"""{game_description}
            Please reply with a creative description of the presidential candidate, {character_name}, in {word_limit} words or less, that emphasizes their personalities. 
            Speak directly to {character_name}.
            Do not add anything else."""
        ),
    ]
    character_description = ChatOpenAI(temperature=1.0)(
        character_specifier_prompt                          # Generate the character description & extract the content
    ).content
    return character_description

# Generate the character headers
def generate_character_header(character_name, character_description): # takes the agent Name and Description
    return f"""{game_description}
Your name is {character_name}.
You are a presidential candidate.
Your description is as follows: {character_description}
You are debating the topic: {topic}.
Your goal is to be as creative as possible and make the voters think you are the best candidate.
"""

# Generate the system messages for the characters
def generate_character_system_message(character_name, character_header):
    return SystemMessage(
        content=(
            f"""{character_header}
You will speak in the style of {character_name}, and exaggerate their personality.
You will come up with creative ideas related to {topic}.
Do not say the same things over and over again.
Speak in the first person from the perspective of {character_name}
For describing your own body movements, wrap your description in '*'.
Do not change roles!
Do not speak from the perspective of anyone else.
Speak only from the perspective of {character_name}.
Stop speaking the moment you finish speaking from your perspective.
Never forget to keep your response to {word_limit} words!
Do not add anything else.
    """
        )
    )

# List Comprenhension: To generate the character descriptions, headers and system messages
character_descriptions = [
    generate_character_description(character_name) for character_name in character_names
]
character_headers = [
    generate_character_header(character_name, character_description)
    for character_name, character_description in zip(
        character_names, character_descriptions
    )
]
character_system_messages = [
    generate_character_system_message(character_name, character_headers)
    for character_name, character_headers in zip(character_names, character_headers)
]

C:\Users\chanho\AppData\Local\Temp\ipykernel_7976\2396525180.py:19: LangChainDeprecationWarning: The method `BaseChatModel.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use :meth:`~invoke` instead.
  character_description = ChatOpenAI(temperature=1.0)(


In [9]:
# Print the character descriptions, headers and system messages
for (
    character_name,
    character_description,
    character_header,
    character_system_message,
) in zip(
    character_names,
    character_descriptions,
    character_headers,
    character_system_messages,
):
    print(f"\n\n{character_name} Description:")
    print(f"\n{character_description}")
    print(f"\n{character_header}")
    print(f"\n{character_system_message.content}")



Donald Trump Description:

Donald Trump: Bold, outspoken, controversial. A divisive figure who thrives in the spotlight and doesn't hold back.

Here is the topic for the presidential debate: Should the US send weapons to Israel?.
The presidential candidates are: Donald Trump, Kamila Harris, Xi Jinping.
Your name is Donald Trump.
You are a presidential candidate.
Your description is as follows: Donald Trump: Bold, outspoken, controversial. A divisive figure who thrives in the spotlight and doesn't hold back.
You are debating the topic: Should the US send weapons to Israel?.
Your goal is to be as creative as possible and make the voters think you are the best candidate.


Here is the topic for the presidential debate: Should the US send weapons to Israel?.
The presidential candidates are: Donald Trump, Kamila Harris, Xi Jinping.
Your name is Donald Trump.
You are a presidential candidate.
Your description is as follows: Donald Trump: Bold, outspoken, controversial. A divisive figure wh

# Setup Bid System

## Output parser for bids
We ask the agents to output a bid to speak. But since the agents are LLMs that output strings, we need to 
1. define a format they will produce their outputs in
2. parse their outputs

We can subclass the [RegexParser](https://github.com/langchain-ai/langchain/blob/master/langchain/output_parsers/regex.py) to implement our own custom output parser for bids.

In [10]:
# Specify the Format for the Bid Output
class BidOutputParser(RegexParser):
    def get_format_instructions(self) -> str:
        return "Your response should be an integer delimited by angled brackets, like this: <int>."

# Parse the Bid Output, extract the bid, and set the default output key
bid_parser = BidOutputParser(
    regex=r"<(\d+)>", 
    output_keys=["bid"], 
    default_output_key="bid"
)

## Generate bidding system message
This is inspired by the prompt used in [Generative Agents](https://arxiv.org/pdf/2304.03442.pdf) for using an LLM to determine the importance of memories. This will use the formatting instructions from our `BidOutputParser`.

In [11]:
# Ask how contradictory the recent message is to the ideas
def generate_character_bidding_template(character_header):
    bidding_template = f"""{character_header}

```
{{message_history}}
```

On the scale of 1 to 10, where 1 is not contradictory and 10 is extremely contradictory, rate how contradictory the following message is to your ideas.

```
{{recent_message}}
```

{bid_parser.get_format_instructions()}
Do nothing else.
    """
    return bidding_template

# Generate the bidding templates for the all characters, iterate through the character headers
character_bidding_templates = [
    generate_character_bidding_template(character_header)
    for character_header in character_headers
]

In [12]:
# Print the bidding templates
for character_name, bidding_template in zip(
    character_names, character_bidding_templates
):
    print(f"{character_name} Bidding Template:")
    print(bidding_template)

Donald Trump Bidding Template:
Here is the topic for the presidential debate: Should the US send weapons to Israel?.
The presidential candidates are: Donald Trump, Kamila Harris, Xi Jinping.
Your name is Donald Trump.
You are a presidential candidate.
Your description is as follows: Donald Trump: Bold, outspoken, controversial. A divisive figure who thrives in the spotlight and doesn't hold back.
You are debating the topic: Should the US send weapons to Israel?.
Your goal is to be as creative as possible and make the voters think you are the best candidate.


```
{message_history}
```

On the scale of 1 to 10, where 1 is not contradictory and 10 is extremely contradictory, rate how contradictory the following message is to your ideas.

```
{recent_message}
```

Your response should be an integer delimited by angled brackets, like this: <int>.
Do nothing else.
    
Kamila Harris Bidding Template:
Here is the topic for the presidential debate: Should the US send weapons to Israel?.
The p

# Task-Specifier

In [13]:
# Task Specifier agent: Refine the debate topic
topic_specifier_prompt = [
    SystemMessage(content="You can make a task more specific."),
    HumanMessage(
        content=f"""{game_description}
        
        You are the debate moderator.
        Please make the debate topic more specific. 
        Frame the debate topic as a problem to be solved.
        Be creative and imaginative.
        Please reply with the specified topic in {word_limit} words or less. 
        Speak directly to the presidential candidates: {*character_names,}.
        Do not add anything else."""
    ),
]
specified_topic = ChatOpenAI(temperature=1.0)(topic_specifier_prompt).content

print(f"Original topic:\n{topic}\n")
print(f"Detailed topic:\n{specified_topic}\n")

Original topic:
Should the US send weapons to Israel?

Detailed topic:
Candidates, how can the US balance supporting Israel's security while promoting peace in the Middle East effectively?



# Define the speaker selection function
Lastly we will define a speaker selection function `select_next_speaker` that takes each agent's bid and selects the agent with the highest bid (with ties broken randomly).

We will define a `ask_for_bid` function that uses the `bid_parser` we defined before to parse the agent's bid. 

We will use `tenacity` to decorate `ask_for_bid` to retry multiple times if the agent's bid doesn't parse correctly and produce a default bid of 0 after the maximum number of tries.

In [14]:
# Create a retrying strategy (useful in scenarios where the bid retrieval process might fail)
@tenacity.retry(
    stop=tenacity.stop_after_attempt(2),
    wait=tenacity.wait_none(),  # No waiting time between retries
    retry=tenacity.retry_if_exception_type(ValueError),
    before_sleep=lambda retry_state: print(
        f"ValueError occurred: {retry_state.outcome.exception()}, retrying..."
    ),
    retry_error_callback=lambda retry_state: 0,
)  # Default value when all retries are exhausted

# Extract the bid from the agent
def ask_for_bid(agent) -> str:
    """
    Ask for agent bid and parses the bid into the correct format.
    """
    bid_string = agent.bid()
    bid = int(bid_parser.parse(bid_string)["bid"])
    return bid

In [15]:
# Select the next speaker
def select_next_speaker(step: int,                              # Takes 1st parameter: current step in the conversation
                        agents: List[DialogueAgent]) -> int:    # Takes 2nd parameter: list of Agents
    
    bids = []
    for agent in agents:            # Loop through the agents and obtain Bid
        bid = ask_for_bid(agent)
        bids.append(bid)

    # Identify the agent with the Highest-Bid, if there is a tie, randomly select one
    max_value = np.max(bids)
    max_indices = np.where(bids == max_value)[0]
    idx = np.random.choice(max_indices)

    # Prints the Bids and the Selected Agent (for Transparency)
    print("Bids:")
    for i, (bid, agent) in enumerate(zip(bids, agents)):
        print(f"\t{agent.name} bid: {bid}")
        if i == idx:
            selected_name = agent.name
    print(f"Selected: {selected_name}")
    print("\n")
    return idx

# Simulate: Main Loop

In [16]:
characters = []

# Create a list of objects for each character
for character_name, character_system_message, bidding_template in zip(
    character_names, character_system_messages, character_bidding_templates
):
    characters.append(
        BiddingDialogueAgent(
            name=character_name,
            system_message=character_system_message,
            model=ChatOpenAI(temperature=0.2),
            bidding_template=bidding_template,
        )
    )

In [17]:
# Sets up and run the simulation
max_iters = 9 # Maximum number of iterations
n = 0 # Initialize the counter

# Initialize the Dialogue Simulator
simulator = DialogueSimulator(agents=characters, selection_function = select_next_speaker)

# Reset the simulator (clear old conversation history) + Inject the specified topic
simulator.reset()
simulator.inject("Debate Moderator", specified_topic)

# Print the specified topic
print(f"(Debate Moderator): {specified_topic}")
print("\n")

# Main Loop
while n < max_iters:
    name, message = simulator.step()
    print(f"({name}): {message}")
    print("\n")
    n += 1

(Debate Moderator): Candidates, how can the US balance supporting Israel's security while promoting peace in the Middle East effectively?


Bids:
	Donald Trump bid: 7
	Kamila Harris bid: 2
	Xi Jinping bid: 5
Selected: Donald Trump


(Donald Trump): Let me tell you, folks, we need to support Israel with the best weapons to ensure peace in the Middle East.


Bids:
	Donald Trump bid: 8
	Kamila Harris bid: 9
	Xi Jinping bid: 8
Selected: Kamila Harris


(Kamila Harris): *As I stand here today, I believe in finding a balance between supporting Israel's security and promoting peace in the Middle East. We must prioritize diplomacy over weapons.*


Bids:
	Donald Trump bid: 9
	Kamila Harris bid: 2
	Xi Jinping bid: 8
Selected: Donald Trump


(Donald Trump): The US must stand strong with Israel and provide them with the weapons they need for security and stability.


Bids:
	Donald Trump bid: 2
	Kamila Harris bid: 8
	Xi Jinping bid: 9
Selected: Xi Jinping


(Xi Jinping): *My fellow candidates, the 